## Imports and paths

In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.covariance import EllipticEnvelope
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
DIR = "Armis_data/"
DEVICES_PATH = DIR+"all_devices.csv"
SESSIONS_PATH = DIR+"all_sessions.csv"

## Read the data

In [4]:
devices = pd.read_csv(DEVICES_PATH)
devices.head()

,Unnamed: 0,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version
0,113,0,1028623,MOBILE_PHONE,Galaxy S8,Samsung,Android,9
1,587,0,48047,MOBILE_PHONE,Galaxy Note 8,Samsung,Android,9
2,668,0,123568,MOBILE_PHONE,H918,LG Electronics,Android,8.0.0
3,830,0,95366,MOBILE_PHONE,iPhone 6,"Apple, Inc.",iOS,NaN
4,886,0,1755023,TABLET,iPad,Apple,iOS,NaN


In [5]:
sessions = pd.read_csv(SESSIONS_PATH)
sessions.head()


,Unnamed: 0,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,...,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev
0,0,0,35,1565074800,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,ecbb92cd941972b779d18451b6f96275587941e4cf07a1...,49152,TCP,790889.0,260,...,93.000000,93.000000,93.000000,93.000000,0.000000,312.6,312.6,312.6,312.6,0.0
1,1,0,35,1565053200,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af0...,49153,TCP,1604622.0,178,...,106.750000,93.400000,94.883333,93.000000,4.317134,318.4,318.4,318.4,318.0,0.0
2,2,0,57,1565082000,e16257c983f2c35d41f39d425651972fa1905e46e968d7...,d43dad76e6cef2231d2efc743e498996b40f8b13fc120b...,443,TCP,NaN,67,...,64.018182,41.000000,44.288312,41.000000,8.700055,NaN,NaN,NaN,NaN,NaN
3,3,0,57,1565082000,1a4f860269acca6c264f00d84c4b63aad00b8f93a77250...,945e37dab8aee93dd4e650f8d17d76a3adfbc6aa70ebba...,443,TCP,NaN,45,...,226.733333,133.571429,194.968236,209.000000,41.777266,NaN,NaN,NaN,NaN,NaN
4,4,0,57,1565082000,df106cbe1ba4a700c00ec8883490f40a8afdb75c15a9ea...,4b43e85e630c2d18a0afaa2a6366367c4fc52d32b4ba5b...,443,TCP,NaN,9,...,185.666667,185.666667,185.666667,185.666667,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
len(devices)

76261

## Generate Feature Set

In [7]:
#select only 5 features: 
#unique hosts count, 
#unique port count, 
#total packets transferred, 
#total inbound bytes 
#total outbound bytes. 



In [8]:
def extract_features(df):
    return df.groupby(["device_id"]).aggregate({"host": "nunique", "port_dst": "nunique", 
                                                "packets_count": "sum", "inbound_bytes_count": "sum", 
                                                "outbound_bytes_count": "sum"})

In [9]:
networks_fs = []
for i in range(4):
    networks_fs.append(extract_features(sessions[sessions.network_id == i]))


## Modeling Fun

In [11]:
#Elliptic Envelope model for our anomaly detection. 
#We use the decision_function in order to receive the confidence and we normalize the values to values between 0 and 1.

In [12]:
# We use the simple min-max normalization in order to normalize the confidence values to 0-1 range.
# Higher score means that this device is probably more anomalous.
def calc_normalized_decision(decision_function_result):
    decision_function_result = -1 * decision_function_result
    minimum = decision_function_result.min()
    maximum = decision_function_result.max()
    return (decision_function_result - minimum) / (maximum - minimum)

In [13]:
def detect_anomalies(feature_set):
    ee = EllipticEnvelope(contamination=0.05).fit(feature_set.values)
    decision_function_result = ee.decision_function(feature_set.values)
    feature_set["confidence"] = calc_normalized_decision(decision_function_result)

In [14]:
for i in range(4):
    detect_anomalies(networks_fs[i])

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.359993842016209 > 71.346665810761721)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.359993842016209 > 71.335668118406417)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.706343967084393 > 71.339096795292804)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.697222156207744 > 71.688483766588348)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: Runtime

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.460404699948981 > 71.439871623188509)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.483174804523543 > 71.449716243883088)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.437395745671239 > 71.417250366005462)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.452846877157356 > 71.447243056379008)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: Runtime

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.559052813581104 > 71.493252165543552)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.547147479606366 > 71.475571623469165)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.547147479606366 > 71.477173535936984)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.683129715547082 > 71.299134453804356)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: Runtime

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.524422851968254 > 71.453185436615044)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.503718379663468 > 71.475391382432591)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.460848739726600 > 71.455135470070331)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.740369524979840 > 71.450601058802874)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: Runtime

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.387660867433326 > 71.383727491646624)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.676765882838922 > 71.438310804611390)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.460848739726600 > 71.448289574752465)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.739375077916890 > 71.456468338403027)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: Runtime

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.443713408292737 > 71.441043821888329)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.359993842016209 > 71.345843768277831)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.465976477278772 > 71.452015693545462)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.359993842016209 > 71.345418080761249)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: Runtime

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.522505525675740 > 71.466798193190257)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (71.547147479606366 > 71.492037704502309)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > pre

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (69.998239611600710 > 69.963626164224848)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (69.923670965782151 > 69.918571955769352)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (69.990077720392790 > 69.949627032286656)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (70.026960338389785 > 70.009213936415975)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: Runtime

/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (61.278513339700687 > 61.277864820285146)
  % (det, previous_det), RuntimeWarning)
/Users/jiminy/anaconda3/lib/python3.7/site-packages/sklearn/covariance/robust_covariance.py:165: RuntimeWarning: Warning! det > previous_det (61.272611257837148 > 61.269051701200908)
  % (det, previous_det), RuntimeWarning)


In [15]:
# Remove the device_id from index and Add network id to each data set
for i in range(4):
    networks_fs[i].reset_index(level=0, inplace=True)
    networks_fs[i]["network_id"] = i

In [16]:
df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
df_to_submit.head()

,network_id,device_id,confidence
0,0,33,0.000178
1,0,35,0.033672
2,0,40,0.224246
3,0,41,0.000550
4,0,53,0.000195


In [17]:
len(df_to_submit)

76038